In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

In [2]:
import copy
import json
import os

import numpy as np

from pycocotools.coco import COCO
from sam3.eval.cgf1_eval import CGF1Evaluator

# SA-Co/Gold

In [3]:
# Update to the directory where the GT annotation and PRED files exist
GT_DIR = # PUT YOUR PATH HERE
PRED_DIR = # PUT YOUR PATH HERE

In [4]:
# Relative file names for GT files for 7 SA-Co/Gold subsets
saco_gold_gts = {
    # MetaCLIP Captioner
    "metaclip_nps": [
            "gold_metaclip_merged_a_release_test.json",
            "gold_metaclip_merged_b_release_test.json",
            "gold_metaclip_merged_c_release_test.json",
    ],
    # SA-1B captioner
    "sa1b_nps": [
            "gold_sa1b_merged_a_release_test.json",
            "gold_sa1b_merged_b_release_test.json",
            "gold_sa1b_merged_c_release_test.json",
    ],
    # Crowded
    "crowded": [
            "gold_crowded_merged_a_release_test.json",
            "gold_crowded_merged_b_release_test.json",
            "gold_crowded_merged_c_release_test.json",
    ],
    # FG Food
    "fg_food": [
            "gold_fg_food_merged_a_release_test.json",
            "gold_fg_food_merged_b_release_test.json",
            "gold_fg_food_merged_c_release_test.json",
    ],
    # FG Sports
    "fg_sports_equipment": [
            "gold_fg_sports_equipment_merged_a_release_test.json",
            "gold_fg_sports_equipment_merged_b_release_test.json",
            "gold_fg_sports_equipment_merged_c_release_test.json",
    ],
    # Attributes
    "attributes": [
            "gold_attributes_merged_a_release_test.json",
            "gold_attributes_merged_b_release_test.json",
            "gold_attributes_merged_c_release_test.json",
    ],
    # Wiki common
    "wiki_common": [
            "gold_wiki_common_merged_a_release_test.json",
            "gold_wiki_common_merged_b_release_test.json",
            "gold_wiki_common_merged_c_release_test.json",
    ],
}

## Run offline evaluation for all 7 SA-Co/Gold subsets

We assume the inference has already been run for all 7 datasets. With the default configurations, the predictions are dumped in a predictable folder

In [5]:
results_gold = {}
results_gold_bbox = {}

for subset_name, gts in saco_gold_gts.items():
    print("Processing subset: ", subset_name)
    gt_paths = [os.path.join(GT_DIR, gt) for gt in gts]
    pred_path = os.path.join(PRED_DIR, f"gold_{subset_name}/dumps/gold_{subset_name}/coco_predictions_segm.json")
    
    evaluator = CGF1Evaluator(gt_path=gt_paths, verbose=True, iou_type="segm") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_segm_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_segm_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_segm_positive_micro_F1"] * 100
    results_gold[subset_name] = cur_results

    # Also eval bbox    
    evaluator = CGF1Evaluator(gt_path=gt_paths, verbose=True, iou_type="bbox") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_bbox_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_bbox_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_bbox_positive_micro_F1"] * 100
    results_gold_bbox[subset_name] = cur_results

Processing subset:  metaclip_nps
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
Loaded 26221 predictions


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33057/33057 [00:10<00:00, 3171.54it/s]


Accumulating results
cgF1 metric, IoU type=segm
 Average cgF1       @[ IoU=0.50:0.95] = 0.473
 Average precision  @[ IoU=0.50:0.95] = 0.609
 Average recall     @[ IoU=0.50:0.95] = 0.532
 Average F1         @[ IoU=0.50:0.95] = 0.568
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.759
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.586
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.652
 Average IL_precision                = 0.916
 Average IL_recall                   = 0.760
 Average IL_F1                       = 0.830
 Average IL_FPR                      = 0.013
 Average IL_MCC                      = 0.807
 Average cgF1       @[ IoU=0.50     ] = 0.568
 Average precision  @[ IoU=0.50     ] = 0.732
 Average recall     @[ IoU=0.50     ] = 0.639
 Average F1         @[ IoU=0.50     ] = 0.682
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.872
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.704
 Average positive_micro_precision @[ IoU=0.50     ] = 0.783
 Average cgF1       @[ IoU=

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33057/33057 [00:08<00:00, 3762.56it/s]


Accumulating results
cgF1 metric, IoU type=bbox
 Average cgF1       @[ IoU=0.50:0.95] = 0.500
 Average precision  @[ IoU=0.50:0.95] = 0.645
 Average recall     @[ IoU=0.50:0.95] = 0.563
 Average F1         @[ IoU=0.50:0.95] = 0.601
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.813
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.620
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.690
 Average IL_precision                = 0.916
 Average IL_recall                   = 0.760
 Average IL_F1                       = 0.831
 Average IL_FPR                      = 0.013
 Average IL_MCC                      = 0.807
 Average cgF1       @[ IoU=0.50     ] = 0.571
 Average precision  @[ IoU=0.50     ] = 0.736
 Average recall     @[ IoU=0.50     ] = 0.642
 Average F1         @[ IoU=0.50     ] = 0.686
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.878
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.707
 Average positive_micro_precision @[ IoU=0.50     ] = 0.787
 Average cgF1       @[ IoU=

...

Accumulating results
cgF1 metric, IoU type=bbox
 Average cgF1       @[ IoU=0.50:0.95] = 0.565
 Average precision  @[ IoU=0.50:0.95] = 0.660
 Average recall     @[ IoU=0.50:0.95] = 0.689
 Average F1         @[ IoU=0.50:0.95] = 0.674
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.901
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.739
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.798
 Average IL_precision                = 0.798
 Average IL_recall                   = 0.819
 Average IL_F1                       = 0.808
 Average IL_FPR                      = 0.048
 Average IL_MCC                      = 0.764
 Average cgF1       @[ IoU=0.50     ] = 0.602
 Average precision  @[ IoU=0.50     ] = 0.703
 Average recall     @[ IoU=0.50     ] = 0.734
 Average F1         @[ IoU=0.50     ] = 0.718
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.934
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.788
 Average positive_micro_precision @[ IoU=0.50     ] = 0.850
 Average cgF1       @[ IoU=

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65452/65452 [00:11<00:00, 5775.55it/s]


Accumulating results
cgF1 metric, IoU type=segm
 Average cgF1       @[ IoU=0.50:0.95] = 0.425
 Average precision  @[ IoU=0.50:0.95] = 0.677
 Average recall     @[ IoU=0.50:0.95] = 0.509
 Average F1         @[ IoU=0.50:0.95] = 0.581
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.811
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.608
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.757
 Average IL_precision                = 0.822
 Average IL_recall                   = 0.607
 Average IL_F1                       = 0.698
 Average IL_FPR                      = 0.004
 Average IL_MCC                      = 0.699
 Average cgF1       @[ IoU=0.50     ] = 0.482
 Average precision  @[ IoU=0.50     ] = 0.767
 Average recall     @[ IoU=0.50     ] = 0.577
 Average F1         @[ IoU=0.50     ] = 0.658
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.905
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.689
 Average positive_micro_precision @[ IoU=0.50     ] = 0.857
 Average cgF1       @[ IoU=

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65452/65452 [00:10<00:00, 6310.26it/s]


Accumulating results
cgF1 metric, IoU type=bbox
 Average cgF1       @[ IoU=0.50:0.95] = 0.443
 Average precision  @[ IoU=0.50:0.95] = 0.706
 Average recall     @[ IoU=0.50:0.95] = 0.529
 Average F1         @[ IoU=0.50:0.95] = 0.605
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.865
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.633
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.788
 Average IL_precision                = 0.822
 Average IL_recall                   = 0.607
 Average IL_F1                       = 0.698
 Average IL_FPR                      = 0.004
 Average IL_MCC                      = 0.699
 Average cgF1       @[ IoU=0.50     ] = 0.483
 Average precision  @[ IoU=0.50     ] = 0.770
 Average recall     @[ IoU=0.50     ] = 0.578
 Average F1         @[ IoU=0.50     ] = 0.660
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.909
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.691
 Average positive_micro_precision @[ IoU=0.50     ] = 0.860
 Average cgF1       @[ IoU=

In [6]:
# Compute averages
METRICS = ["cgf1", "il_mcc", "pmf1"]
avg_stats, avg_stats_bbox = {}, {}
for key in METRICS:
    avg_stats[key] = sum(res[key] for res in results_gold.values()) / len(results_gold)
    avg_stats_bbox[key] = sum(res[key] for res in results_gold_bbox.values()) / len(results_gold_bbox)
results_gold["Average"] = avg_stats
results_gold_bbox["Average"] = avg_stats_bbox

In [7]:
# Pretty print segmentation results
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_gold:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_gold[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))

In [8]:
# Pretty print bbox detection results
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_gold:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_gold_bbox[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))

# SA-Co/Silver

In [9]:
# Update to the directory where the GT annotation and PRED files exist
GT_DIR =  # PUT YOUR PATH HERE
PRED_DIR =  # PUT YOUR PATH HERE

In [10]:
saco_silver_gts = {
    "bdd100k": "silver_bdd100k_merged_test.json",
    "droid": "silver_droid_merged_test.json",
    "ego4d": "silver_ego4d_merged_test.json",
    "food_rec": "silver_food_rec_merged_test.json",
    "geode": "silver_geode_merged_test.json",
    "inaturalist": "silver_inaturalist_merged_test.json",
    "nga_art": "silver_nga_art_merged_test.json",
    "sav": "silver_sav_merged_test.json",
    "yt1b": "silver_yt1b_merged_test.json",
    "fathomnet": "silver_fathomnet_test.json",
}

In [11]:
results_silver = {}
results_silver_bbox = {}

for subset_name, gt in saco_silver_gts.items():
    print("Processing subset: ", subset_name)
    gt_path = os.path.join(GT_DIR, gt)
    pred_path = os.path.join(PRED_DIR, f"silver_{subset_name}/dumps/silver_{subset_name}/coco_predictions_segm.json")
    
    evaluator = CGF1Evaluator(gt_path=gt_path, verbose=True, iou_type="segm") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_segm_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_segm_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_segm_positive_micro_F1"] * 100
    results_silver[subset_name] = cur_results

    # Also eval bbox    
    evaluator = CGF1Evaluator(gt_path=gt_path, verbose=True, iou_type="bbox") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_bbox_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_bbox_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_bbox_positive_micro_F1"] * 100
    results_silver_bbox[subset_name] = cur_results


Processing subset:  bdd100k
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
Loaded 31278 predictions


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5439/5439 [00:01<00:00, 3496.20it/s]


Accumulating results
cgF1 metric, IoU type=segm
 Average cgF1       @[ IoU=0.50:0.95] = 0.466
 Average precision  @[ IoU=0.50:0.95] = 0.514
 Average recall     @[ IoU=0.50:0.95] = 0.644
 Average F1         @[ IoU=0.50:0.95] = 0.572
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.669
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.601
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.564
 Average IL_precision                = 0.870
 Average IL_recall                   = 0.952
 Average IL_F1                       = 0.909
 Average IL_FPR                      = 0.196
 Average IL_MCC                      = 0.775
 Average cgF1       @[ IoU=0.50     ] = 0.563
 Average precision  @[ IoU=0.50     ] = 0.621
 Average recall     @[ IoU=0.50     ] = 0.779
 Average F1         @[ IoU=0.50     ] = 0.691
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.769
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.726
 Average positive_micro_precision @[ IoU=0.50     ] = 0.681
 Average cgF1       @[ IoU=

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5439/5439 [00:01<00:00, 4945.36it/s]


Accumulating results
cgF1 metric, IoU type=bbox
 Average cgF1       @[ IoU=0.50:0.95] = 0.462
 Average precision  @[ IoU=0.50:0.95] = 0.510
 Average recall     @[ IoU=0.50:0.95] = 0.639
 Average F1         @[ IoU=0.50:0.95] = 0.567
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.673
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.596
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.559
 Average IL_precision                = 0.870
 Average IL_recall                   = 0.952
 Average IL_F1                       = 0.909
 Average IL_FPR                      = 0.196
 Average IL_MCC                      = 0.775
 Average cgF1       @[ IoU=0.50     ] = 0.562
 Average precision  @[ IoU=0.50     ] = 0.620
 Average recall     @[ IoU=0.50     ] = 0.777
 Average F1         @[ IoU=0.50     ] = 0.689
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.769
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.725
 Average positive_micro_precision @[ IoU=0.50     ] = 0.679
 Average cgF1       @[ IoU=

...

Accumulating results
cgF1 metric, IoU type=bbox
 Average cgF1       @[ IoU=0.50:0.95] = 0.424
 Average precision  @[ IoU=0.50:0.95] = 0.549
 Average recall     @[ IoU=0.50:0.95] = 0.571
 Average F1         @[ IoU=0.50:0.95] = 0.560
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.732
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.588
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.605
 Average IL_precision                = 0.853
 Average IL_recall                   = 0.841
 Average IL_F1                       = 0.847
 Average IL_FPR                      = 0.121
 Average IL_MCC                      = 0.721
 Average cgF1       @[ IoU=0.50     ] = 0.502
 Average precision  @[ IoU=0.50     ] = 0.651
 Average recall     @[ IoU=0.50     ] = 0.677
 Average F1         @[ IoU=0.50     ] = 0.664
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.818
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.697
 Average positive_micro_precision @[ IoU=0.50     ] = 0.718
 Average cgF1       @[ IoU=

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 281205/281205 [00:14<00:00, 19028.75it/s]


Accumulating results
cgF1 metric, IoU type=segm
 Average cgF1       @[ IoU=0.50:0.95] = 0.515
 Average precision  @[ IoU=0.50:0.95] = 0.472
 Average recall     @[ IoU=0.50:0.95] = 0.711
 Average F1         @[ IoU=0.50:0.95] = 0.567
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.690
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.600
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.519
 Average IL_precision                = 0.839
 Average IL_recall                   = 0.885
 Average IL_F1                       = 0.861
 Average IL_FPR                      = 0.003
 Average IL_MCC                      = 0.859
 Average cgF1       @[ IoU=0.50     ] = 0.615
 Average precision  @[ IoU=0.50     ] = 0.564
 Average recall     @[ IoU=0.50     ] = 0.848
 Average F1         @[ IoU=0.50     ] = 0.677
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.828
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.716
 Average positive_micro_precision @[ IoU=0.50     ] = 0.619
 Average cgF1       @[ IoU=

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 281205/281205 [00:14<00:00, 19965.61it/s]


Accumulating results
cgF1 metric, IoU type=bbox
 Average cgF1       @[ IoU=0.50:0.95] = 0.540
 Average precision  @[ IoU=0.50:0.95] = 0.495
 Average recall     @[ IoU=0.50:0.95] = 0.745
 Average F1         @[ IoU=0.50:0.95] = 0.595
 Average positive_macro_F1 @[ IoU=0.50:0.95] = 0.747
 Average positive_micro_F1 @[ IoU=0.50:0.95] = 0.629
 Average positive_micro_precision @[ IoU=0.50:0.95] = 0.544
 Average IL_precision                = 0.839
 Average IL_recall                   = 0.885
 Average IL_F1                       = 0.861
 Average IL_FPR                      = 0.003
 Average IL_MCC                      = 0.859
 Average cgF1       @[ IoU=0.50     ] = 0.620
 Average precision  @[ IoU=0.50     ] = 0.568
 Average recall     @[ IoU=0.50     ] = 0.855
 Average F1         @[ IoU=0.50     ] = 0.683
 Average positive_macro_F1 @[ IoU=0.50     ] = 0.842
 Average positive_micro_F1 @[ IoU=0.50     ] = 0.722
 Average positive_micro_precision @[ IoU=0.50     ] = 0.625
 Average cgF1       @[ IoU=

In [12]:
# Compute averages
METRICS = ["cgf1", "il_mcc", "pmf1"]
avg_stats, avg_stats_bbox = {}, {}
for key in METRICS:
    avg_stats[key] = sum(res[key] for res in results_silver.values()) / len(results_silver)
    avg_stats_bbox[key] = sum(res[key] for res in results_silver_bbox.values()) / len(results_silver_bbox)
results_silver["Average"] = avg_stats
results_silver_bbox["Average"] = avg_stats_bbox

In [13]:
# Pretty print segmentation results
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_silver:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_silver[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))

In [14]:
# Pretty print bbox detection results
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_silver_bbox:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_silver_bbox[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))